# Planning Pattern

The agent autonomously plans a sequence of tasks before executing them.

<img src = "https://github.com/neural-maze/agentic_patterns/raw/main/img/planning_pattern.png" height = "600">

In [112]:
import os 
import re
import json
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
from groq import Groq
from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.utils.extraction import extract_tag_content

model = "llama-3.3-70b-versatile"
client = Groq()


In [113]:
react_system_prompt = """
You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
%s
</tools>

Example session:

<question>What's the current temperature in Madrid?</question>
<thought>I need to get the current weather in Madrid</thought>
<tool_call>{"name": "get_current_weather","arguments": {"location": "Madrid", "unit": "celsius"}, "id": 0}</tool_call>

You will be called again with this:

<observation>{0: {"temperature": 25, "unit": "celsius"}}</observation>

You then output:

<response>The current temperature in Madrid is 25 degrees Celsius</response>

Additional constraints:

- If the user asks you something unrelated to any of the tools above, answer freely enclosing your answer with <response></response> tags.

"""

In [114]:
import math
@tool
def sum_elements(a:int,b:int)-> int:
    return a+b

@tool
def multiply_elements(a:int , b:int) ->int:
    return a*b

@tool
def compute_log(x:int) -> float|str:
    if x<=0:
        return "Logarithm is undefined for values less than or equal to 0"
    return math.log(x)


tools = {
    "sum_elements": sum_elements,
    "multiply_elements" : multiply_elements,
    "compute_log" : compute_log
}

In [115]:
print(f"Tool Name : {sum_elements.name}")
print(F"Tool Signature : {sum_elements.fn_signature}")

Tool Name : sum_elements
Tool Signature : {"name": "sum_elements", "description": null, "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}}


In [116]:
tools_signature = sum_elements.fn_signature + ",\n" + multiply_elements.fn_signature + ",\n" + compute_log.fn_signature
tools_signature

'{"name": "sum_elements", "description": null, "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},\n{"name": "multiply_elements", "description": null, "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},\n{"name": "compute_log", "description": null, "parameters": {"properties": {"x": {"type": "int"}}}}'

In [117]:
react_system_prompt = react_system_prompt % tools_signature
print(react_system_prompt)


You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
{"name": "sum_elements", "description": null, "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},
{"name": "multiply_elements", "description": null, "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},
{"name": "compute_log", "des

## ReAct Loop -1 

In [118]:
user_qn = "I want to calculate the sum of 1234  and 5678 and multiply the result by 5.Then i want to take the logarithm of this result"

chat_history = [
    {
        "role" : "system",
        "content" : react_system_prompt
    },
    {
        "role" : "user" ,
        "content" : user_qn
    }
]

In [119]:
response = client.chat.completions.create(
    messages= chat_history,
    model=model
)

response = response.choices[0].message.content
print(response)

<thought>I need to calculate the sum of 1234 and 5678, then multiply the result by 5, and finally take the logarithm of this result. I will use the sum_elements, multiply_elements, and compute_log functions to achieve this.</thought>
<tool_call>{"name": "sum_elements","arguments": {"a": 1234, "b": 5678}, "id": 0}</tool_call>


In [120]:
chat_history.append(
    {
        "role" : "assistant" ,
        "content" : response
    }
)

## ReAct Loop -2

In [121]:
tool_call = extract_tag_content(response ,tag = "tool_call")
tool_call

TagContentResult(content=['{"name": "sum_elements","arguments": {"a": 1234, "b": 5678}, "id": 0}'], found=True)

In [122]:
tool_call = json.loads(tool_call.content[0])
tool_call

{'name': 'sum_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 0}

In [123]:
tool_result = tools[tool_call["name"]].run(**tool_call["arguments"])

In [124]:
tool_result

6912

In [125]:
assert tool_result == 1234 + 5678

In [126]:
chat_history.append(
    {
        "role" : "user" ,
        "content" : f"<Observation> {tool_result} </observation> "
    }
)

## ReAct Loop -3 

In [127]:
response = client.chat.completions.create(
    messages= chat_history,
    model=model
)

response = response.choices[0].message.content
print(response)

<thought>I have the sum of 1234 and 5678, which is 6912. Now, I need to multiply this result by 5.</thought>
<tool_call>{"name": "multiply_elements","arguments": {"a": 6912, "b": 5}, "id": 1}</tool_call>


In [128]:
chat_history.append(
    {
        "role" : "assistant" ,
        "content" : response
    }
)

## ReAct Loop - 4

In [129]:
tool_call = extract_tag_content(response ,tag = "tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = tools[tool_call["name"]].run(**tool_call["arguments"])
print(tool_result)

34560


In [130]:
assert tool_result == (1234+5678) *5 

In [131]:
chat_history.append(
    {
        "role" : "user" ,
        "content" : f"<Observation> {tool_result} </observation> "
    }
)

## ReAct Loop - 5

In [132]:
response = client.chat.completions.create(
    messages= chat_history,
    model=model
)

response = response.choices[0].message.content
print(response)

<thought>I have the result of multiplying 6912 by 5, which is 34560. Now, I need to calculate the logarithm of this result.</thought>
<tool_call>{"name": "compute_log","arguments": {"x": 34560}, "id": 2}</tool_call>


In [133]:
chat_history.append(
    {
        "role" : "assistant" ,
        "content" : response
    }
)

## ReAct Loop - 6

In [134]:
tool_call = extract_tag_content(response ,tag = "tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = tools[tool_call["name"]].run(**tool_call["arguments"])
print(tool_result)

10.450452222917992


In [135]:
assert tool_result == math.log((1234 + 5678) * 5)

In [136]:
chat_history.append(
    {
        "role" : "user" ,
        "content" : f"<Observation> {tool_result} </observation> "
    }
)

## ReAct Loop - 7

In [137]:
response = client.chat.completions.create(
    messages= chat_history,
    model=model
)

response = response.choices[0].message.content
print(response)

<response>The result of calculating the sum of 1234 and 5678, then multiplying by 5, and finally taking the logarithm is 10.450452222917992</response>


## Agentic Pattern Library

In [142]:
from agentic_patterns.planning_pattern.react_agent import ReactAgent

agent = ReactAgent(tools = [sum_elements,multiply_elements,compute_log],model=model)


In [144]:
response = agent.run(user_msg = "I want to calculate the sum of 1234  and 5678 and multiply the result by 5.Then i want to take the logarithm of this result")
print(response)


Thought: I need to calculate the sum of 1234 and 5678, then multiply the result by 5, and finally compute the logarithm of this result. I will use the sum_elements, multiply_elements, and compute_log functions to achieve this.

Using Tool: sum_elements

Tool call dict: 
{'name': 'sum_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 0}

Tool result: 
6912

Observations: {0: 6912}

Thought: To multiply 6912 by 5, I will use the multiply_elements function.

Using Tool: multiply_elements

Tool call dict: 
{'name': 'multiply_elements', 'arguments': {'a': 6912, 'b': 5}, 'id': 1}

Tool result: 
34560

Observations: {1: 34560}

Thought: To compute the logarithm of 34560, I will use the compute_log function.

Using Tool: compute_log

Tool call dict: 
{'name': 'compute_log', 'arguments': {'x': 34560}, 'id': 2}

Tool result: 
10.450452222917992

Observations: {2: 10.450452222917992}
The sum of 1234 and 5678 is 6912. Multiplying this result by 5 gives 34560. The logarithm of 34560 is approxi